In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
filename = "data/total_comm.csv"
df_comments_orig = pd.read_csv(filename)
filename = "data/total_post.csv"
df_posts_orig = pd.read_csv(filename)

/var/folders/ks/5_cxdzt52gz2lghcr64x80zm0000gn/T/ipykernel_15641/858612826.py:2: DtypeWarning: Columns (0,1,2,3,5,12,15,19,20,22,24,25,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,64,65,68,69,72,74,76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comments_orig = pd.read_csv(filename)
/var/folders/ks/5_cxdzt52gz2lghcr64x80zm0000gn/T/ipykernel_15641/858612826.py:4: DtypeWarning: Columns (0,2,4,6,7,8,9,10,11,12,16,17,19,20,21,22,23,25,27,28,29,30,32,35,36,39,41,44,45,46,49,50,55,61,63,64,67,70,71,75,78,82,83,84,85,90,93,97,99,100,101,102,104,105,106,107,108,109,111,113,114,116,117,118,120,123,124,125,127) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posts_orig = pd.read_csv(filename)


In [5]:
# Drop all Nan values.
df_comments = df_comments_orig.dropna(subset=['link_id']).copy()

In [45]:
flair_df = pd.concat([df_comments[['author', 'author_flair_css_class']], df_posts_orig[['author', 'author_flair_css_class']]])

# Keeps users with mtbi flag
mbti = ['intj', 'intp', 'entj', 'entp', 'infj', 'infp', 'enfj', 'enfp', 'istj', 'isfj', 'estj', 'esfj', 'istp', 
        'isfp', 'estp', 'esfp']
flair_df = flair_df[flair_df['author_flair_css_class'].isin(mbti)]

# Keeps the first letter
flair_df['author_flair_css_class'] = flair_df['author_flair_css_class'].apply(lambda x: x[0])
print(flair_df['author_flair_css_class'].value_counts())

# Dictionary-> each author is mapped to a list of all his flags
flair_dict = flair_df.groupby('author')['author_flair_css_class'].apply(list).to_dict()

i    1220601
e     623125
Name: author_flair_css_class, dtype: int64


In [35]:
# Convert created_utc columns to datetime
df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'], unit='s')
df_posts_orig['created_utc'] = pd.to_datetime(df_posts_orig['created_utc'], unit='s' , errors='coerce')

combined_df = pd.concat([df_comments[['author', 'created_utc']], df_posts_orig[['author', 'created_utc']]])

# Group by author and calculate the first and last activity date
user_activity = combined_df.groupby('author')['created_utc'].agg(['min', 'max'])

# Calculate the duration of activity in days
user_activity['duration'] = (user_activity['max'] - user_activity['min']).dt.days

# Filter users active for at least 14 days
active_users = user_activity[user_activity['duration'] >= 14].index

In [37]:
import networkx as nx

G = nx.MultiDiGraph()
G.add_nodes_from(active_users)

active_comments = df_comments[df_comments['author'].isin(active_users)]

# Add edges for interactions
for idx, comment in active_comments.iterrows():
    if idx%1000 == 0:
        print(f'{round(idx/len(active_comments)*100)} % done')
    author = comment['author']
    parent_id = comment['parent_id']
    parent_type, parent_name = parent_id.split('_')
    
    if parent_type == 't1':  # Comment interaction
        parent_comment = df_comments[df_comments['id'] == parent_name]
        if not parent_comment.empty:
            parent_author = parent_comment.iloc[0]['author']
            if parent_author in active_users:
                G.add_edge(author, parent_author)
    elif parent_type == 't3':  # Post interaction
        parent_post = df_posts_orig[df_posts_orig['id'] == parent_name]
        if not parent_post.empty:
            parent_author = parent_post.iloc[0]['author']
            if parent_author in active_users:
                G.add_edge(author, parent_author)

0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done
0 % done


KeyboardInterrupt: 

In [ ]:
nx.write_gexf(G, 'data/interaction_graph.gexf')